In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.misc import imread
import matplotlib.image as mpimg
import os

In [2]:
def load_all_labels(data_path):
    raw_labels = []
    with open(data_path) as input_labels:
        for line in input_labels:
            tokens = line.strip().split(',')
            raw_labels.append(tokens[1])
    return raw_labels

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
raw_labels = load_all_labels('data/train_v2.csv')
raw_labels = raw_labels[1:]
label_tf = CountVectorizer(binary=True, max_features=20)
labels = label_tf.fit_transform(raw_labels)

In [4]:
labels

<40479x17 sparse matrix of type '<class 'numpy.int64'>'
	with 116205 stored elements in Compressed Sparse Row format>

In [5]:
n_samples = labels.shape[0]
indices = np.random.permutation(n_samples)
#Number of images for train dataset and test dataset
n_train = 15000
n_test = 4000
# split the train and test 
#n_test = int(n_samples * 0.1)
print("Numbers of total images: ", n_samples)
print("Numbers of training images: ", n_train)
print("Number of test images: ", n_test)
#test = train_img[indices[:n_test]]
#train = train_img[indices[n_test:]]
test_labels = labels[indices[:n_test]]
train_labels = labels[indices[n_test:n_train+n_test]]

Numbers of total images:  40479
Numbers of training images:  15000
Number of test images:  4000


In [6]:
def load_image(data_path, indices, image_size):
    num_images = 0
    dataset = np.ndarray(shape=(len(indices), image_size, image_size, 3),dtype=np.float32)
    for i in tqdm(range(len(indices)), total = len(indices)):
        image = data_path + 'train_' + str(indices[i]) + '.jpg'
        image_data = mpimg.imread(image)[:,:,:3]
        #Normalization
        image_data = np.multiply(image_data, 1.0/255.0)
        dataset[num_images, :, :, :] = image_data
        num_images += 1
    print('Full dataset tensor:', dataset.shape)
    return dataset

In [7]:
data_path = "data/train-jpg/"
image_size = 256
print("Loading training images data...")
train = load_image(data_path, indices[n_test:n_train+n_test], image_size)
print("Loading testing images data...")
test = load_image(data_path, indices[:n_test], image_size)

Loading training images data...


100%|████████████████████████████████████████████████████████████████████████████| 15000/15000 [08:31<00:00, 29.35it/s]


Full dataset tensor: (15000, 256, 256, 3)
Loading testing images data...


100%|██████████████████████████████████████████████████████████████████████████████| 4000/4000 [02:02<00:00, 38.02it/s]


Full dataset tensor: (4000, 256, 256, 3)


In [8]:
print(train.shape)
print(train_labels.shape)
print(test.shape)
print(test_labels.shape)

(15000, 256, 256, 3)
(15000, 17)
(4000, 256, 256, 3)
(4000, 17)


In [9]:
# Compute KL divergence
test_disb = np.sum(test_labels, axis=0)
train_disb = np.sum(train_labels, axis=0)
test_disb = test_disb / float(np.sum(test_disb))
train_disb = train_disb / float(np.sum(train_disb))

kl = np.sum(np.multiply(train_disb, (np.log(train_disb) - np.log(test_disb))))
print(kl)

kl = np.sum(np.multiply(test_disb, (np.log(test_disb) - np.log(train_disb))))
print(kl)

0.000947299816648
0.00093818038732


In [10]:
import os
import tensorflow as tf

#os.environ["CUDA_VISIBLE_DEVICES"]="3"

def get_session(gpu_fraction=0.1):
    '''Assume that you have 6GB of GPU memory and want to allocate ~2GB'''
    num_threads = os.environ.get('OMP_NUM_THREADS')
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_fraction)

    if num_threads:
        return tf.Session(config=tf.ConfigProto(
            gpu_options=gpu_options, intra_op_parallelism_threads=num_threads))
    else:
        return tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
    
import keras.backend as K
#K.set_session(get_session(1.0))

Using TensorFlow backend.


In [11]:
import tensorflow as tf
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.applications import vgg16
from keras.models import Model

img_width = 256
img_height = 256
batch_size = 32
num_classes = 17
epochs = 30
data_augmentation = False

x_train = train
x_test = test
y_train = train_labels.toarray()
y_test = test_labels.toarray()

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (15000, 256, 256, 3)
15000 train samples
4000 test samples


In [12]:
stat = np.array(np.sum(y_train, axis = 0)).flatten()
print(stat)
mu = 0.15
scores = [np.log(mu*x_train.shape[0]/x) for x in stat]
class_weights = {}
for i in range(len(scores)):
    class_weights[i] = scores[i] if scores[i]>1.0 else 1.0
print(class_weights)

[ 4637   122   293   115    32 10589   743    38  1666  1410  1007  2661
 13934  3057   142    73  2773]
{0: 1.0, 1: 2.9146644504652093, 2: 2.0385128861813984, 3: 2.9737533668352159, 4: 4.252949592398739, 5: 1.0, 6: 1.1079894504807066, 7: 4.0810993354720804, 8: 1.0, 9: 1.0, 10: 1.0, 11: 1.0, 12: 1.0, 13: 1.0, 14: 2.7628584375972052, 15: 3.4282260540500746, 16: 1.0}


In [14]:
stat = np.array(np.sum(y_train, axis = 0)).flatten()
print(stat)
mu = 1
scores = [mu*np.sum(stat)/x for x in stat]
class_weights = {}
for i in range(len(scores)):
    class_weights[i] = scores[i] #if scores[i]>1.0 else 1.0
print(class_weights)

[ 4637   122   293   115    32 10589   743    38  1666  1410  1007  2661
 13934  3057   142    73  2773]
{0: 3.2348501186111709, 1: 122.95081967213115, 2: 51.194539249146757, 3: 130.43478260869566, 4: 468.75, 5: 1.4165643592407215, 6: 20.188425302826378, 7: 394.73684210526318, 8: 9.0036014405762312, 9: 10.638297872340425, 10: 14.895729890764647, 11: 5.636978579481398, 12: 1.0765035165781542, 13: 4.9067713444553487, 14: 105.63380281690141, 15: 205.47945205479451, 16: 5.4093040028849622}


In [15]:
#build the VGG16 network
base_model = vgg16.VGG16(weights = None, include_top = False, input_shape = (img_height, img_width, 3))
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0         
__________

In [16]:
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation = 'relu')(x)
x = Dropout(0.25)(x)
x = Dense(num_classes, activation = 'sigmoid')(x)
model = Model(inputs = base_model.input, outputs = x)

In [17]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [18]:
def accuracy_with_threshold(y_true, y_pred):
    threshold = 0.5
    y_pred = K.cast(K.greater(y_pred, threshold), K.floatx())
    return K.mean(K.equal(y_true, y_pred))

def hamming_dist(y_true, y_pred):
    threshold = 0.5
    y_pred = K.cast(K.greater(y_pred, threshold), K.floatx())
    return K.mean(K.sum(K.abs(y_true - y_pred), axis=1))

In [29]:
def f_score(y_true, y_pred):
    threshold = 0.5
    y_pred = K.greater(y_pred, threshold)
    
    y_pred = tf.to_int32(y_pred)
    y_true = tf.to_int32(y_true)
    like = tf.ones_like(y_true)
    
    num_retrieves = tf.reduce_sum(y_pred, 1)
    
    num_relevants = tf.logical_and(tf.equal(y_true,like),tf.equal(y_pred,like))
    num_relevants = tf.to_int32(num_relevants)                                
    num_relevants = tf.reduce_sum(num_relevants,1)
    
    total_relevants = tf.reduce_sum(y_true, 1)
    
    precision = tf.div(tf.to_float(num_relevants), tf.to_float(num_retrieves))
    recall = tf.div(tf.to_float(num_relevants), tf.to_float(total_relevants))
    #beta = 2.
    #beta_sqr = beta * beta
    #f1 = tf.scalar_mul((1 + beta_sqr), tf.div(tf.multiply(precision,recall),tf.add(tf.scalar_mul(beta_sqr, precision), recall)))
    return tf.reduce_mean(recall) 

In [ ]:
# Let's train the model using RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=[accuracy_with_threshold, hamming_dist, f_score])


if not data_augmentation:
    print('Not using data augmentation.')
    #hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,validation_data=(x_test, y_test), class_weight=class_weights, verbose=2, shuffle=True)
    hist = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs,
                     validation_data=(x_test, y_test), shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test))

Not using data augmentation.
Train on 15000 samples, validate on 4000 samples
Epoch 1/30


 2592/15000 [====>.........................] - ETA: 1859s - loss: 0.1094 - accuracy_with_threshold: 0.9596 - hamming_dist: 0.6875 - f_score: 0.856 - ETA: 1451s - loss: 0.1935 - accuracy_with_threshold: 0.9338 - hamming_dist: 1.1250 - f_score: 0.798 - ETA: 1149s - loss: 0.2038 - accuracy_with_threshold: 0.9301 - hamming_dist: 1.1875 - f_score: 0.775 - ETA: 1011s - loss: 0.1962 - accuracy_with_threshold: 0.9324 - hamming_dist: 1.1484 - f_score: 0.771 - ETA: 908s - loss: 0.1951 - accuracy_with_threshold: 0.9342 - hamming_dist: 1.1187 - f_score: 0.768 - ETA: 833s - loss: 0.2131 - accuracy_with_threshold: 0.9317 - hamming_dist: 1.1615 - f_score: 0.77 - ETA: 799s - loss: 0.2082 - accuracy_with_threshold: 0.9322 - hamming_dist: 1.1518 - f_score: 0.77 - ETA: 760s - loss: 0.2027 - accuracy_with_threshold: 0.9336 - hamming_dist: 1.1289 - f_score: 0.77 - ETA: 742s - loss: 0.2014 - accuracy_with_threshold: 0.9328 - hamming_dist: 1.1424 - f_score: 0.76 - ETA: 728s - loss: 0.1977 - accuracy_with_thr

 5216/15000 [=========>....................] - ETA: 486s - loss: 0.1617 - accuracy_with_threshold: 0.9373 - hamming_dist: 1.0667 - f_score: 0.78 - ETA: 484s - loss: 0.1613 - accuracy_with_threshold: 0.9375 - hamming_dist: 1.0629 - f_score: 0.78 - ETA: 482s - loss: 0.1615 - accuracy_with_threshold: 0.9376 - hamming_dist: 1.0614 - f_score: 0.78 - ETA: 480s - loss: 0.1616 - accuracy_with_threshold: 0.9375 - hamming_dist: 1.0618 - f_score: 0.78 - ETA: 479s - loss: 0.1617 - accuracy_with_threshold: 0.9376 - hamming_dist: 1.0614 - f_score: 0.78 - ETA: 477s - loss: 0.1617 - accuracy_with_threshold: 0.9375 - hamming_dist: 1.0625 - f_score: 0.78 - ETA: 474s - loss: 0.1618 - accuracy_with_threshold: 0.9374 - hamming_dist: 1.0639 - f_score: 0.78 - ETA: 473s - loss: 0.1618 - accuracy_with_threshold: 0.9374 - hamming_dist: 1.0650 - f_score: 0.78 - ETA: 472s - loss: 0.1616 - accuracy_with_threshold: 0.9374 - hamming_dist: 1.0649 - f_score: 0.78 - ETA: 470s - loss: 0.1614 - accuracy_with_threshold: 0

 7840/15000 [==============>...............] - ETA: 408s - loss: 0.1609 - accuracy_with_threshold: 0.9371 - hamming_dist: 1.0688 - f_score: 0.78 - ETA: 407s - loss: 0.1607 - accuracy_with_threshold: 0.9372 - hamming_dist: 1.0670 - f_score: 0.78 - ETA: 405s - loss: 0.1604 - accuracy_with_threshold: 0.9373 - hamming_dist: 1.0651 - f_score: 0.78 - ETA: 404s - loss: 0.1601 - accuracy_with_threshold: 0.9375 - hamming_dist: 1.0632 - f_score: 0.78 - ETA: 402s - loss: 0.1600 - accuracy_with_threshold: 0.9375 - hamming_dist: 1.0618 - f_score: 0.78 - ETA: 400s - loss: 0.1599 - accuracy_with_threshold: 0.9375 - hamming_dist: 1.0618 - f_score: 0.78 - ETA: 399s - loss: 0.1596 - accuracy_with_threshold: 0.9377 - hamming_dist: 1.0594 - f_score: 0.78 - ETA: 397s - loss: 0.1594 - accuracy_with_threshold: 0.9378 - hamming_dist: 1.0570 - f_score: 0.78 - ETA: 395s - loss: 0.1598 - accuracy_with_threshold: 0.9377 - hamming_dist: 1.0598 - f_score: 0.78 - ETA: 394s - loss: 0.1597 - accuracy_with_threshold: 0

10464/15000 [===================>..........] - ETA: 288s - loss: 0.1571 - accuracy_with_threshold: 0.9381 - hamming_dist: 1.0518 - f_score: 0.78 - ETA: 286s - loss: 0.1570 - accuracy_with_threshold: 0.9381 - hamming_dist: 1.0521 - f_score: 0.78 - ETA: 285s - loss: 0.1568 - accuracy_with_threshold: 0.9382 - hamming_dist: 1.0509 - f_score: 0.78 - ETA: 283s - loss: 0.1567 - accuracy_with_threshold: 0.9383 - hamming_dist: 1.0491 - f_score: 0.78 - ETA: 282s - loss: 0.1567 - accuracy_with_threshold: 0.9383 - hamming_dist: 1.0497 - f_score: 0.78 - ETA: 281s - loss: 0.1567 - accuracy_with_threshold: 0.9383 - hamming_dist: 1.0494 - f_score: 0.78 - ETA: 279s - loss: 0.1567 - accuracy_with_threshold: 0.9383 - hamming_dist: 1.0497 - f_score: 0.78 - ETA: 278s - loss: 0.1565 - accuracy_with_threshold: 0.9383 - hamming_dist: 1.0487 - f_score: 0.78 - ETA: 277s - loss: 0.1564 - accuracy_with_threshold: 0.9383 - hamming_dist: 1.0481 - f_score: 0.78 - ETA: 275s - loss: 0.1563 - accuracy_with_threshold: 0

...

 5248/15000 [=========>....................] - ETA: 518s - loss: 0.1160 - accuracy_with_threshold: 0.9556 - hamming_dist: 0.7556 - f_score: 0.86 - ETA: 516s - loss: 0.1156 - accuracy_with_threshold: 0.9558 - hamming_dist: 0.7519 - f_score: 0.86 - ETA: 513s - loss: 0.1150 - accuracy_with_threshold: 0.9560 - hamming_dist: 0.7482 - f_score: 0.86 - ETA: 511s - loss: 0.1152 - accuracy_with_threshold: 0.9559 - hamming_dist: 0.7493 - f_score: 0.86 - ETA: 509s - loss: 0.1158 - accuracy_with_threshold: 0.9556 - hamming_dist: 0.7540 - f_score: 0.86 - ETA: 507s - loss: 0.1159 - accuracy_with_threshold: 0.9555 - hamming_dist: 0.7557 - f_score: 0.86 - ETA: 507s - loss: 0.1162 - accuracy_with_threshold: 0.9554 - hamming_dist: 0.7581 - f_score: 0.86 - ETA: 506s - loss: 0.1161 - accuracy_with_threshold: 0.9553 - hamming_dist: 0.7594 - f_score: 0.86 - ETA: 504s - loss: 0.1160 - accuracy_with_threshold: 0.9554 - hamming_dist: 0.7576 - f_score: 0.86 - ETA: 501s - loss: 0.1157 - accuracy_with_threshold: 0

 7872/15000 [==============>...............] - ETA: 407s - loss: 0.1181 - accuracy_with_threshold: 0.9546 - hamming_dist: 0.7720 - f_score: 0.85 - ETA: 406s - loss: 0.1180 - accuracy_with_threshold: 0.9546 - hamming_dist: 0.7713 - f_score: 0.85 - ETA: 404s - loss: 0.1179 - accuracy_with_threshold: 0.9546 - hamming_dist: 0.7711 - f_score: 0.85 - ETA: 403s - loss: 0.1179 - accuracy_with_threshold: 0.9546 - hamming_dist: 0.7712 - f_score: 0.85 - ETA: 401s - loss: 0.1176 - accuracy_with_threshold: 0.9547 - hamming_dist: 0.7702 - f_score: 0.85 - ETA: 399s - loss: 0.1176 - accuracy_with_threshold: 0.9547 - hamming_dist: 0.7708 - f_score: 0.85 - ETA: 398s - loss: 0.1176 - accuracy_with_threshold: 0.9547 - hamming_dist: 0.7707 - f_score: 0.85 - ETA: 396s - loss: 0.1176 - accuracy_with_threshold: 0.9547 - hamming_dist: 0.7702 - f_score: 0.85 - ETA: 395s - loss: 0.1177 - accuracy_with_threshold: 0.9546 - hamming_dist: 0.7713 - f_score: 0.85 - ETA: 393s - loss: 0.1177 - accuracy_with_threshold: 0

10496/15000 [===================>..........] - ETA: 292s - loss: 0.1180 - accuracy_with_threshold: 0.9543 - hamming_dist: 0.7767 - f_score: 0.85 - ETA: 291s - loss: 0.1181 - accuracy_with_threshold: 0.9543 - hamming_dist: 0.7770 - f_score: 0.85 - ETA: 289s - loss: 0.1182 - accuracy_with_threshold: 0.9542 - hamming_dist: 0.7781 - f_score: 0.85 - ETA: 288s - loss: 0.1180 - accuracy_with_threshold: 0.9543 - hamming_dist: 0.7769 - f_score: 0.85 - ETA: 287s - loss: 0.1178 - accuracy_with_threshold: 0.9543 - hamming_dist: 0.7761 - f_score: 0.85 - ETA: 285s - loss: 0.1178 - accuracy_with_threshold: 0.9544 - hamming_dist: 0.7759 - f_score: 0.85 - ETA: 284s - loss: 0.1179 - accuracy_with_threshold: 0.9543 - hamming_dist: 0.7762 - f_score: 0.85 - ETA: 283s - loss: 0.1179 - accuracy_with_threshold: 0.9544 - hamming_dist: 0.7756 - f_score: 0.85 - ETA: 281s - loss: 0.1178 - accuracy_with_threshold: 0.9544 - hamming_dist: 0.7760 - f_score: 0.85 - ETA: 280s - loss: 0.1179 - accuracy_with_threshold: 0

13120/15000 [=========================>....] - ETA: 185s - loss: 0.1183 - accuracy_with_threshold: 0.9541 - hamming_dist: 0.7798 - f_score: 0.85 - ETA: 184s - loss: 0.1184 - accuracy_with_threshold: 0.9541 - hamming_dist: 0.7803 - f_score: 0.85 - ETA: 183s - loss: 0.1184 - accuracy_with_threshold: 0.9541 - hamming_dist: 0.7798 - f_score: 0.85 - ETA: 181s - loss: 0.1184 - accuracy_with_threshold: 0.9541 - hamming_dist: 0.7795 - f_score: 0.85 - ETA: 180s - loss: 0.1184 - accuracy_with_threshold: 0.9541 - hamming_dist: 0.7795 - f_score: 0.85 - ETA: 179s - loss: 0.1183 - accuracy_with_threshold: 0.9542 - hamming_dist: 0.7787 - f_score: 0.85 - ETA: 177s - loss: 0.1182 - accuracy_with_threshold: 0.9542 - hamming_dist: 0.7782 - f_score: 0.85 - ETA: 176s - loss: 0.1184 - accuracy_with_threshold: 0.9542 - hamming_dist: 0.7788 - f_score: 0.85 - ETA: 174s - loss: 0.1182 - accuracy_with_threshold: 0.9543 - hamming_dist: 0.7776 - f_score: 0.85 - ETA: 173s - loss: 0.1183 - accuracy_with_threshold: 0

14592/15000 [============================>.] - ETA: 79s - loss: 0.1192 - accuracy_with_threshold: 0.9540 - hamming_dist: 0.7828 - f_score: 0.853 - ETA: 77s - loss: 0.1192 - accuracy_with_threshold: 0.9539 - hamming_dist: 0.7829 - f_score: 0.853 - ETA: 76s - loss: 0.1193 - accuracy_with_threshold: 0.9539 - hamming_dist: 0.7834 - f_score: 0.853 - ETA: 74s - loss: 0.1194 - accuracy_with_threshold: 0.9539 - hamming_dist: 0.7839 - f_score: 0.853 - ETA: 73s - loss: 0.1195 - accuracy_with_threshold: 0.9539 - hamming_dist: 0.7840 - f_score: 0.853 - ETA: 72s - loss: 0.1194 - accuracy_with_threshold: 0.9539 - hamming_dist: 0.7835 - f_score: 0.853 - ETA: 70s - loss: 0.1194 - accuracy_with_threshold: 0.9539 - hamming_dist: 0.7836 - f_score: 0.853 - ETA: 70s - loss: 0.1195 - accuracy_with_threshold: 0.9539 - hamming_dist: 0.7839 - f_score: 0.853 - ETA: 68s - loss: 0.1195 - accuracy_with_threshold: 0.9539 - hamming_dist: 0.7845 - f_score: 0.853 - ETA: 67s - loss: 0.1196 - accuracy_with_threshold: 0.

In [ ]:
print(hist.history)

In [ ]:
model.save('submission/submission2/planet2_8.h5')

In [ ]:
pred = model.predict(x_test, batch_size=batch_size, verbose=0)
y_pred = (pred >= 0.5).astype(np.int32)
y_pred = y_pred.astype(np.int32)
y_test = y_test.astype(np.int32)
num_relevants = np.sum(y_test & y_pred, axis=1)
num_retrieves = np.sum(y_pred, axis=1)
total_relevants = np.sum(y_test, axis=1)

# precision
precision = np.divide(num_relevants.astype(np.float), num_retrieves.astype(np.float))
precision = np.nan_to_num(precision)
# recall
recall = np.divide(num_relevants.astype(np.float), total_relevants.astype(np.float))
recall = np.nan_to_num(recall)
beta = 2.
beta_sqr = beta * beta
f1 = (1 + beta_sqr) * np.divide((precision * recall),(beta_sqr * precision + recall))
f1 = np.nan_to_num(f1)
print('precision={}'.format(np.nanmean(precision)))
print('recall={}'.format(np.nanmean(recall)))
print('f1={}'.format(np.nanmean(f1)))

In [25]:
def get_confusion_matrix(y_pred,y_test):
    confusion_matrix = np.zeros(shape=(18,18),dtype=np.int16)
    for i in range(y_test.shape[0]):
        for j in range(17):
            if y_test[i][j] == 1 and y_pred[i][j] == 1:
                confusion_matrix[j+1][j+1] += 1
            if y_test[i][j] == 1 and y_pred[i][j] == 0:
                confusion_matrix[j+1][0] += 1
            if y_test[i][j] == 0 and y_pred[i][j] == 1:
                confusion_matrix[0][j+1] += 1
                
    return confusion_matrix

In [26]:
confusion_matrix = get_confusion_matrix(y_pred,y_test)

In [ ]:
confusion_matrix = pd.DataFrame(confusion_matrix)

In [ ]:
confusion_matrix.columns = ['NULL','agriculture',
 'artisinal_mine',
 'bare_ground',
 'blooming',
 'blow_down',
 'clear',
 'cloudy',
 'conventional_mine',
 'cultivation',
 'habitation',
 'haze',
 'partly_cloudy',
 'primary',
 'road',
 'selective_logging',
 'slash_burn',
 'water']

In [62]:
pre = model.predict(x_test,batch_size=32)

In [63]:
pre.shape

(1000, 17)

In [68]:
pre[0]

array([ 0.21239533,  0.00969336,  0.02384911,  0.05670618,  0.00668827,
        0.84235227,  0.01966204,  0.00752299,  0.19295782,  0.08035329,
        0.02582694,  0.1628222 ,  0.96848643,  0.14170928,  0.05123476,
        0.02263625,  0.18485051], dtype=float32)

In [73]:
y_pred = K.cast(K.greater(pre, 0.5), K.floatx())

In [74]:
y_pred

<tf.Tensor 'Cast_6:0' shape=(1000, 17) dtype=float32>

In [76]:
sess = tf.InteractiveSession()
y_pred = y_pred.eval()

In [78]:
y_pred[0]

array([ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.], dtype=float32)

In [80]:
#for i in range(1000):
#    for j in range(17):
#        if pre[i][j] >=0.5:
#            pre[i][j] = 1
#        else:
#            pre[i][j]=0

In [82]:
accracy = K.mean(K.equal(y_test, y_pred)).eval()

In [83]:
accracy

0.92405879